<a href="https://colab.research.google.com/github/charlesdethibault/Marketing/blob/main/hdyhau_openAI_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Before you start, Make sure you upload two csv files in your directory which is the folder icon on your left. Please make sure the names of the files are similar
**note: if you would like to use google sheets instead, let me know**
* "openaikey.csv" , containing your openAIkey in cell A1
* "hdyhau.csv", with all responses in column A with column title "hdyhau"

# 2. Once, files are there, click on the runtime tab above, then 'run all"

# 3. If you do not see the "hdyhau_categorized.csv" file created, refresh your directory using the refresh directory icon.

# 4. hover on the "hdyhau_categorized.csv" file and click on the 3 dots to download the file

In [1]:
# install / import libraries
!pip install openai
import openai

!pip install docx2txt
import docx2txt

import os
import pandas as pd
pd.set_option('display.max_colwidth', 500)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=45acbc2ce9f9e53384fb77fd61f1c505c9363ea66ffdf34f6b4e12e6d00fcaf5
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [2]:
# get APIkey
my_openAIkey = ', '.join(list(pd.read_csv('openaikey.csv')))
os.environ['OPENAI_API_KEY'] =my_openAIkey
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
#load data from directory
data = pd.read_csv('hdyhau.csv')
list_hdyhau = list(data['hdyhau'])

In [4]:
#print the top 10 items of your file

data.head(10)

,hdyhau
0,I came across your website while doing some research for my job search
1,"Sarah Johnson, a former colleague, recommended your company to me"
2,I found your job posting on Indeed
3,"My friend, James Lee, suggested I check out your website"
4,I discovered your company through a Google search for tech startups in the area
5,I saw your company's booth at a career fair and decided to learn more
6,Your company was recommended to me by my career counselor
7,I came across your company while browsing Glassdoor
8,I heard about your company from a recruiter who reached out to me on LinkedIn
9,Your company was featured in an article I read on TechCrunch


In [5]:
#create prompt
intro_prompt ='we ask the prospects how they heard about us. Below are their responses, can you provide a category for each response in the below list. The categories should be Referral, Social Media, Online research, Podcast, Event, Online ads, blog, newsletters, and others. I also would like to list the name of the source: whom referred, which website, what podcast. The output structure should the "prospect freext - Category name - Source name" '
print(intro_prompt)
prompt = intro_prompt + str(list_hdyhau)

we ask the prospects how they heard about us. Below are their responses, can you provide a category for each response in the below list. The categories should be Referral, Social Media, Online research, Podcast, Event, Online ads, blog, newsletters, and others. I also would like to list the name of the source: whom referred, which website, what podcast. The output structure should the "prospect freext - Category name - Source name" 


In [6]:
# send request
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=prompt,
  temperature=0,
  max_tokens=2484,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
r = (response["choices"][0])
print(r["text"])



I came across your website while doing some research for my job search - Online Research - Website 
Sarah Johnson, a former colleague, recommended your company to me - Referral - Sarah Johnson 
I found your job posting on Indeed - Online Ads - Indeed 
My friend, James Lee, suggested I check out your website - Referral - James Lee 
I discovered your company through a Google search for tech startups in the area - Online Research - Google 
I saw your company's booth at a career fair and decided to learn more - Event - Career Fair 
Your company was recommended to me by my career counselor - Referral - Career Counselor 
I came across your company while browsing Glassdoor - Online Research - Glassdoor 
I heard about your company from a recruiter who reached out to me on LinkedIn - Referral - Recruiter 
Your company was featured in an article I read on TechCrunch - Online Research - TechCrunch 
I saw your ad on Facebook and decided to check out your website - Online Ads - Facebook 
My frien

In [7]:
#save file
data_lines = r["text"].split('\n')
data_rows = [line.split(' - ') for line in data_lines]
df = pd.DataFrame(data_rows, columns=['response', 'channel', 'source'])
df.to_csv('hdyhau_categorized')
df.head(10)

,reponse,channel,source
0,,None,None
1,,None,None
2,I came across your website while doing some research for my job search,Online Research,Website
3,"Sarah Johnson, a former colleague, recommended your company to me",Referral,Sarah Johnson
4,I found your job posting on Indeed,Online Ads,Indeed
5,"My friend, James Lee, suggested I check out your website",Referral,James Lee
6,I discovered your company through a Google search for tech startups in the area,Online Research,Google
7,I saw your company's booth at a career fair and decided to learn more,Event,Career Fair
8,Your company was recommended to me by my career counselor,Referral,Career Counselor
9,I came across your company while browsing Glassdoor,Online Research,Glassdoor
